# **<font color = '#3498eb'>SECTOR & COMPANY</font>**

Extreme Sports House is a mountain and adventure distribution company. 

The outdoor apparel and gear sector, which encompasses mountain and adventure clothing and objects, has seen significant growth over the years. According to reports by the Outdoor Industry Association, consumer spending on outdoor recreation totaled $887 billion in 2017. This growth has been fueled by an increasing consumer focus on health and well-being, as well as a surge in adventure tourism. Various trends, such as the rising popularity of sustainable and eco-friendly products, are shaping the industry.

# **<font color = '#3498eb'>OBJETIVES</font>**

The company has enlisted me to:

· Extract maximum value from their existing data.

· Identify the most profitable channels.

· Determine who their most valuable clients are.

· Develop strategies to increase profit margins.

· Review the product portfolio to cut costs.

· Optimize customer management processes.

· Build a SQL-based recommendation system to increase cross-selling opportunities.

# **<font color = '#3498eb'>LIBRARIES</font>**

In [ ]:
%pip install tabulate
%pip install mysql-connector-python
%pip install pymysql

In [15]:
import pandas as pd
import pymysql
import tabulate 

# **<font color = '#3498eb'>DATA LOADING AND UNDERSTANDING</font>**

In [12]:
conn = pymysql.connect(host='localhost', user='root', password='b8SO0ibY5claaghlgiehgld', database='extremesportshouse')

eshdb_cursor = conn.cursor()

In [14]:
eshdb_cursor.execute("SELECT DATABASE();")

fetched_results = eshdb_cursor.fetchone()
print("Currently using the database:", fetched_results[0])

Currently using the database: extremesportshouse


In [29]:
# I review the tables of the database and then review each table.

eshdb_cursor.execute("show tables from extremesportshouse;")

print(tabulate.tabulate([list(eachtuple) for eachtuple in eshdb_cursor.fetchall()]))

--------------------
canales
productos
tiendas
v_ventas_agr_pedido
v_ventas_agr_pedido2
ventas
ventas_agr
--------------------


In [30]:
eshdb_cursor.execute("select * from canales;")

print(tabulate.tabulate([list(eachtuple) for eachtuple in eshdb_cursor.fetchall()]))

--  -----------
 1  Fax
 2  Telephone
 3  Mail
 4  E-mail
 5  Web
 6  Sales visit
 7  Special
 8  Other
 9  Other
10  Other
11  Other
12  Other
--  -----------


In [39]:
eshdb_cursor.execute("select * from productos;")
column_names = [desc[0] for desc in eshdb_cursor.description]

print(tabulate.tabulate([list(row) for row in eshdb_cursor.fetchall()[:10]], headers=column_names))

  id_prod  linea              tipo          producto                   marca      color          coste    precio
---------  -----------------  ------------  -------------------------  ---------  -----------  -------  --------
     1110  Camping Equipment  Cooking Gear  TrailChef Water Bag        TrailChef  Clear           2.77      6.59
     2110  Camping Equipment  Cooking Gear  TrailChef Canteen          TrailChef  Brown           6.92     12.92
     3110  Camping Equipment  Cooking Gear  TrailChef Kitchen Kit      TrailChef  Unspecified    15.78     23.8
     4110  Camping Equipment  Cooking Gear  TrailChef Cup              TrailChef  Silver          0.85      3.66
     5110  Camping Equipment  Cooking Gear  TrailChef Cook Set         TrailChef  Silver         34.41     54.93
     6110  Camping Equipment  Cooking Gear  TrailChef Deluxe Cook Set  TrailChef  Silver         78.72    129.72
     7110  Camping Equipment  Cooking Gear  TrailChef Single Flame     TrailChef  Silver         

# **<font color = '#3498eb'>DATA QUALITY</font>**